# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

## Vignesh Madharapakkam Pagadala

## Summary
In this Notebook, we implement the following algorithms:
1. Negamax
2. Negamax with Iterative-Deepening
3. Negamax with Iterative-Deepening and Alpha-Beta Pruning

We create a class for a Tic-Tac-Toe game, and try out these methodologies with different Tic-Tac-Toe games. Additionally, we build a function using which we compare and analyze the results.

We also implement another game to try the algorithms on, and synthesize a new opponent for playing Tic-Tac-Toe with.

# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening) 
	* [Functions Implemented](#I.-Functions-Implemented) <br><br>
	* [Negamax](#II.-Negamax)
        * [Overview](#II.a.-Overview)
        * [Algorithm Definition](#II.b.-Algorithm-Definition)
        * [Implementation](#II.c.-Implementation)
        * [Applying Negamax to play Tic-Tac-Toe](#II.d.-Applying-Negamax-to-play-Tic-Tac-Toe)
        <br><br>
	* [Negamax with Iterative Deepening](#III.-Negamax-with-Iterative-Deepening)
	* [Negamax with Iterative Deepening and Alpha-Beta Pruning](#IV.-Negamax-with-Iterative-Deepening-and-Alpha-Beta-Pruning)<br><br>
	* [Comparision](#V.-Comparision)
        * [Function to calculate the Effective Branching Factor](#V.a.-Function-to-calculate-the-Effective-Branching-Factor)
        * [Function to print results in a comparable manner](#V.b.-Function-to-print-results-in-a-comparable-manner) <br><br>
    * [Extra Credit Problems](#VI.-Extra-Credit-Problems)
        * [A New Game - Adding to Fifty Apples](#VI.a.-A-New-Game:-Adding-to-Fifty-Apples)
            * [Game Description](#i.-Game-Description)
            * [Implementation of Game Class](#ii.-Implementation-of-Game-Class)
            * [Opponent Definition](#iii.-Opponent-Definition)
            * [Function to play the game](#iv.-Function-to-play-the-game) <br><br>
        * [Random Move-Choosing Opponent](#VI.b.-Random-Move-Choosing-Opponent)
            * [Implementation of Opponent](#i.-Implementation-of-Opponent)
            * [Win Rate of Player X against this opponent ](#ii.-Win-Rate-of-Player-X-against-this-opponent)

## I. Functions Implemented in this Notebook
* [negamax](#negamax)
* [playGame](#playGame)
* [opponent](#opponent)
* [negamaxIDS](#negamaxIDS)
* [negamaxab](#negamaxab)
* [negamaxIDSab](#negamaxIDSab)
* [ebf](#ebf)
* [playGames](#playGames)
* [playGames2](#playGames2)
* [opponent2](#opponent2)
* [opponent3](#opponent3)
* [winRate](#winRate)

## II. Negamax
### II.a. Overview
Negamax is basically a variant of Minimax search with some alterations made, such that the maximum child node is always chosen at each player's play, unlike in Minimax where the maximizer and minimizer nodes alternatively select the maximum and minimum child node respectively. In Negamax, we bank on the zero-sum property of a two-player game (i.e., the reward/punishment for each player at the end of a game is equal and opposite). Therefore, it becomes clear that the use of Negamax is restricted to those turn-based, two-player games which satisfy the zero-sum property. Since max(a,b) = -min(-a, -b), we perform only maximization at each player's turn if we negate the value returned, and the utility values are also negated between alternating players.

### II.b. Algorithm Definition
#### Source: Breuker, Dennis M. Memory versus Search in Games, Maastricht University, October 16, 1998

### II.c. Implementation

#### negamax

In [9]:
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

### II.d. Applying Negamax to play Tic-Tac-Toe
For playing Tic-Tac-Toe using Negamax, we create the following class, which defines various functions specific for Tic-Tac-Toe.

#### Class TTT 

In [10]:
class TTT(object):
    def __init__(self):
        self.movesExplored = 0
        self.board = [' ']*9
        self.player = 'X'
        self.playerLookAHead = self.player
    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]
    def getMoves(self):
        moves = self.locations(' ')
        return moves
    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None 
    def isOver(self):
        return self.getUtility() is not None
    def makeMove(self, move):
        self.movesExplored += 1
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player
    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'
    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s
    def getNumberMovesExplored(self):
        return self.movesExplored
    def getWinningValue(self):
        return 1

We now have the following function *playGame* which runs the game.

#### playGame

In [11]:
def playGame(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
    print('Number of moves explored', game.getNumberMovesExplored())

The function *opponent* will essentially act as the opponent in this game, but it uses the silly strategy of playing the first available position in the board.

#### opponent

In [12]:
def opponent(board):
    return board.index(' ')

Testing out *playGame*.

In [13]:
game = TTT()
playGame(game,opponent,20)

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 
Number of moves explored 558334


## III. Negamax with Iterative-Deepening
In the following cells we shall implement a function *negamaxIDS* which implements Negamax with Iterative-Deepening Search (IDS). In this function, multiple Negamax searches are performed for depths 1, 2, 3, ..., *maxDepth* (here *maxDepth* is the maximum depth of the graph which we specify). If a winning move is found before the maximum depth is reached, then IDS is halted. In the case of Tic-Tac-Toe, this winning value is 1.   

#### negamaxIDS

In [14]:
def negamaxIDS(game, maxDepth):
    # Iterate maxDepth+1 times
    for depth in range(maxDepth + 1):
        # Call negamax for each depth value.
        result, move = negamax(game, depth)
        # Get the winning value. 
        winVal = game.getWinningValue()
        # If we arrive at the winning value, then break out and return the same.
        if result == winVal:
            break
    return result, move

In [15]:
game = TTT()
negamaxIDS(game,5)

(1, 0)

## IV. Negamax with Iterative-Deepening and Alpha-Beta Pruning

#### negamaxab

In [16]:
def negamaxab(game, depthLeft, alpha = -float('inf'), beta = float('inf')):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamaxab(game, depthLeft-1, -beta, -alpha)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        alpha = max(bestValue, alpha)
        if bestValue >= beta:
            return bestValue, bestMove
    return bestValue, bestMove

#### negamaxIDSab

In [17]:
def negamaxIDSab(game, maxDepth):
    for depth in range(maxDepth + 1):
        #print("We're at depth: ", depth)
        # CALLING negamax FOR EACH DEPTH VALUE
        result, move = negamaxab(game, depth)
        winVal = game.getWinningValue()
        if result == winVal:
            return result, move
    return result, move

Let's try out *negamaxab* and *negamaxIDSab*.

In [18]:
negamaxab(game, 5)

(1, 0)

In [19]:
negamaxIDSab(game, 5)

(1, 0)

## V. Comparision
We shall now compare the different algorithms implemented in this Notebook, and observe the differences between them. For this purpose, the function *playGames* is constructed. This functions essentially runs a game for each algorithm and displays the following information:
1. Number of nodes visited by the algorithm
2. Number of moves made by X
3. Depth of the game
4. Effective Branching Factor

### V.a. Function to calculate the Effective Branching Factor

#### ebf

In [20]:
# This function computes the Effective Branching Factor (EBF) for a tree with 'nNodes' nodes and depth 'depth'. 
# We use Binary Search to compute the EBF.
def ebf(nNodes, depth, precision = 0.01):
    # Initialize low and high values.
    left = 1
    right = nNodes
    # If no nodes in tree return 0
    if nNodes == 0:
        return 0
    # Keep iterating till the difference between left and right is lesser than the specified precision.
    b = 1.0
    while(True):
        # If the difference is lesser than the precision, then break out.
        if abs(right - left) <= precision:
            break
        # Get the middle value
        b = (left + right)/2
        # Solve equation to find n
        n = (pow(b, (depth+1)) - 1)/(b - 1)
        if nNodes < n:
            right = b - precision
        elif nNodes > n:
            left = b + precision
        else:
            return b
    return b    

### V.b. Function to print results in a comparable manner
The following cell contains the implementation for the *playGames* function which takes in the following parameters:
1. opponent: A function which generates moves for the opponent.
2. depthLimit: The maximum depth of the search tree, to which the algorithm can explore.
3. funList: The list of functions which represent different search algorithms to be used to play the game.

For each function specified in *funList*, *playGames* plays a complete Tic-Tac-Toe game. After all the games have been played (with all the algorithms), the above mentioned data is displayed side-by-side, for our analysis.  

#### playGames 

In [25]:
def playGames(opponent,depthLimit, funlist):
    # Create an empty list to maintain a list of strings which are to be printed at the ned.
    strlst = []
    for fun in funlist:
        print()
        print(fun.__name__, ':')
        game = TTT()
        print(game)
        while not game.isOver():
            score,move = fun(game,depthLimit)
            if move == None :
                print('move is None. Stopping.')
                break
            game.makeMove(move)
            print('Player', game.player, 'to', move, 'for score' ,score)
            print(game)
            if not game.isOver():
                game.changePlayer()
                opponentMove = opponent(game.board)
                #print(opponentMove)
                game.makeMove(opponentMove)
                print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
                print(game)
                game.changePlayer()
        # Get depth
        xl = len(game.locations('X'))
        ol = len(game.locations('O'))
        dep =  xl + ol
        strlst.append(fun.__name__ + ' made ' + str(len(game.locations('X'))) + ' moves. ' + str(game.getNumberMovesExplored()) + ' moves explored for ebf(' + str(game.getNumberMovesExplored()) + ', ' + str(dep) + ') of ' + str(round(ebf(game.getNumberMovesExplored(), dep), 2)))
    for s in strlst:
        print(s)

In [26]:
playGames(opponent, 10, [negamax, negamaxIDS, negamaxIDSab])


negamax :
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 

negamaxIDS :
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 

negamaxIDSab :
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 
n

## Grader Result

In [27]:
%run -i A4grader.py


Testing negamax starting from ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of 1

--- 10/10 points. negamax correctly explored 124 states.

Testing negamax starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of -1 and move of 5

Testing negamaxIDS with max depth of 5, starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamaxIDS correctly returns value of -1 and move of 5

Testing negamaxIDSab starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 20/20 points. negamaxIDSab correctly returns value of -1 and move of 5

Testing playGame with opponent that always plays in highest numbered position.
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 8
X| | 
-----
 | | 
-----
 | |O
Player X to 2 for score 1
X| |X
-----
 | | 
-----
 | |O
Player O to 7
X| |X
-----
 | | 
-----
 |O|O
Player X to 1 for 

## VI. Extra Credit Problems

### VI.a. A New Game: Adding to Fifty Apples

#### i. Game Description
We have two players, A and B. Each player can choose between 1 and 9 (inclusive) apples to add to a bag (which is empty initially) and take turns in doing so. This bag can hold a maximum of 50 apples only. 

The objective of each player is to try and be the last person to insert an apple/apples into the bag. The player who inserts into the bag last is considered to have won the game.

#### ii. Implementation of Game Class
The class *addingToFifty* represents the class for the above game. In this class, we define various functions specific to this game, which shall be explained in more detail through comments in the cell below.

In [28]:
class addingToFifty(object):
    # Constructor which initializes various parameters. 'movesExplore' represents the counter maintained for counting the 
    # number of nodes visited by the algorithm. 'board' represents the bag with a certain number of apples. 'player' is the
    # current player. 'acount' and 'bcount' are used to count the number of moves made by A and B respectively.
    def __init__(self):
        self.movesExplored = 0
        self.board = 0
        self.player = 'A'
        self.playerLookAHead = self.player
        self.acount = 0
        self.bcount = 0 
        
    # Returns the available moves. 
    def getMoves(self):
        # Choose a number of apples between 1 and 9. 
        moves = range(1,10)
        moves = list(moves)
        # If adding any of these moves to bag gets the number of apples to be more than 50, then remove said move.
        for i in moves:
            n = self.board + i
            if n > 50:
                # Remove this move from moves list (since it's invalid).
                moves.remove(i)
        return moves
    
    # Returns the utility value. 
    def getUtility(self):
        # Terminating condition
        if self.board == 50:
            # A wins
            if self.playerLookAHead == 'A':
                return 1
            # B wins
            else:
                return -1
        # No draws can occur in this game.
        else:
            return None 
    
    # To check if the game is over.
    def isOver(self):
        return self.getUtility() is not None

    # Apply a given move onto a given state of the game.
    def makeMove(self, move):
        # Increment moves counter.
        self.movesExplored += 1
        # Update the 'bag' based on move i.e. add the chosen number to board.
        self.board += move
        # Switch players after a play
        self.playerLookAHead = 'A' if self.playerLookAHead == 'B' else 'B'
    
    # Function to change players.
    def changePlayer(self):
        self.player = 'A' if self.player == 'B' else 'B'
        self.playerLookAHead = self.player

    # Function to undo a move. 
    def unmakeMove(self, move):
        # To undo, we just need to subtract the previously added number to board.
        self.board -= move
        self.playerLookAHead = 'A' if self.playerLookAHead == 'B' else 'B'

    # To print the game.
    def __str__(self):
        return str(self.board)
    
    # Returns total number of nodes visited.
    def getNumberMovesExplored(self):
        return self.movesExplored
    
    # The winning value for this game. 
    def getWinningValue(self):
        # In this game, 1 is the winning value.
        return 1

    # Function to increment the moves made by A and B.
    def incCount(self):
        if self.playerLookAHead == 'A':
            self.acount += 1
        else:
            self.bcount += 1

#### iii. Opponent Definition
Here we define an opponent which chooses a random number of apples to insert into the bag.

#### opponent2

In [29]:
import random as rd
def opponent2(board):
    
    diff = 50 - board
    # If it's possible to insert more apples than the maximum capacity of the bag, then choose pseudo-random number such that
    # the maximum capacity of the bag (50) is not exceeded.
    if diff <= 8:
        r = rd.randint(1,diff)
    # Otherwise, just select pseudo-random number between 1 and 9 (inclusive).
    else:
        r = rd.randint(1, 9)
    return r

#### iv. Function to play the game
The function *playGames2* is similar to the *playGames* implementation above. It's essentially been modified a little to play the 'Adding to Fifty Apples' game specified here, with the different algorithms. 

#### playGames2

In [30]:
def playGames2(opponent, depthLimit, funlist):
    # Create an empty list to maintain a list of strings which are to be printed at the end.
    strlst = []
    for fun in funlist: # Iterate through the functions provided in funlist.
        # Print function name first
        print()
        print(fun.__name__, ':')
        
        # Instantiate an object of the addingToFifty class.
        game = addingToFifty()
        print(game)
        
        # Loop till game isn't over
        while not game.isOver():
            
            # Call the function (algorithm).
            score,move = fun(game,depthLimit)
            
            # If depthLimit specified is insufficient to find a solution.
            if move == None :
                print('move is None. Stopping.')
                break
            # Increment acount and bcount
            game.incCount()
            # Commit move
            game.makeMove(move)
            # Print the move made and the player's incentive.
            print('Player', game.player, 'to', move, 'for score' ,score)
            print(game)
            if not game.isOver():
                # If game still isn't over, switch players and let opponent make the next move.
                game.changePlayer()
                opponentMove = opponent(game.board)
                # Commit opponent's move
                game.makeMove(opponentMove)
                # Print move made by opponent.
                print('Player', game.player, 'to', opponentMove)
                print(game)
                # After opponent is done, switch players again.
                game.changePlayer()
        # Get depth
        dep =  game.acount + game.bcount
        # Keep appending results string for each function. 
        strlst.append(fun.__name__ + ' made ' + str(game.acount) + ' moves. ' + str(game.getNumberMovesExplored()) + ' moves explored for ebf(' + str(game.getNumberMovesExplored()) + ', ' + str(dep) + ') of ' + str(round(ebf(game.getNumberMovesExplored(), dep), 2)))
    # Print the results side by side.
    for s in strlst:
        print(s)

In [33]:
playGames2(opponent2, 6, [negamax, negamaxIDS, negamaxIDSab])


negamax :
0
Player A to 5 for score 1
5
Player B to 2
7
Player A to 1 for score 1
8
Player B to 6
14
Player A to 1 for score 1
15
Player B to 4
19
Player A to 1 for score 1
20
Player B to 6
26
Player A to 1 for score 1
27
Player B to 9
36
Player A to 1 for score 1
37
Player B to 5
42
Player A to 1 for score 1
43
Player B to 4
47
Player A to 1 for score 1
48
Player B to 1
49
Player A to 1 for score 1
50

negamaxIDS :
0
Player A to 5 for score 1
5
Player B to 2
7
Player A to 7 for score 1
14
Player B to 6
20
Player A to 3 for score 1
23
Player B to 2
25
Player A to 7 for score 1
32
Player B to 6
38
Player A to 3 for score 1
41
Player B to 9
50

negamaxIDSab :
0
Player A to 5 for score 1
5
Player B to 8
13
Player A to 1 for score 1
14
Player B to 8
22
Player A to 1 for score 1
23
Player B to 6
29
Player A to 3 for score 1
32
Player B to 4
36
Player A to 5 for score 1
41
Player B to 5
46
Player A to 4 for score 1
50
negamax made 9 moves. 2483623 moves explored for ebf(2483623, 9) of 5.01


### VI.b. Random Move-Choosing Opponent

#### i. Implementation of Opponent
In the following cell, we define a new opponent with the function *opponent3* for playing Tic-Tac-Toe with. This opponent has been designed to make random moves across the board. 

#### opponent3

In [34]:
def opponent3(board):
    # Find out empty positions in the game board.
    moves = [i for i, mark in enumerate(board) if mark == ' ']
    # Generate random integer. This shall be used as the index for 'moves' list, thereby producing a random move.
    r = rd.randint(0, len(moves)-1)
    return moves[r]

Let's try playing a game with this opponent. 

In [36]:
playGames(opponent3, 10, [negamaxIDSab])


negamaxIDSab :
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 5
X| | 
-----
 | |O
-----
 | | 
Player X to 1 for score 1
X|X| 
-----
 | |O
-----
 | | 
Player O to 2
X|X|O
-----
 | |O
-----
 | | 
Player X to 8 for score 1
X|X|O
-----
 | |O
-----
 | |X
Player O to 3
X|X|O
-----
O| |O
-----
 | |X
Player X to 4 for score 1
X|X|O
-----
O|X|O
-----
 | |X
negamaxIDSab made 4 moves. 6085 moves explored for ebf(6085, 7) of 3.29


#### ii. Win Rate of Player X against this opponent 
Let's try to find out the number of times X wins over this opponent, as an average over multiple games. For doing this, we define the function below. It takes in an opponent function (*opponent*), *depthLimit*, a list of algorithms to try (*funlist*) and *n*, through which we can specify the number of games to play.

The function computes the average number of games X has won over the total number of games played and displays it as a percentage. 

#### winRate

In [37]:
def winRate(opponent, depthLimit, funlist, n):
    # Iterate through funlist
    for fun in funlist:
        # Print the name of the function being tried out currently.
        print()
        print('Now we are trying out function: ', fun.__name__)
        # Maintain a list to append the score of each game that is played.
        scorelist = []
        # Loop to play 'n' games.
        for i in range(n):
            # Instantiate TTT object
            game = TTT()
            while not game.isOver():
                # Call function (algorithm)
                score,move = fun(game,depthLimit)
                # If depthLimit is not sufficient
                if move == None :
                    print('Insufficient depth specified. Stopping game!')
                    break
                # Apply move
                game.makeMove(move)
                if not game.isOver():
                    # Change player and let opponent make move.
                    game.changePlayer()
                    opponentMove = opponent(game.board)
                    game.makeMove(opponentMove)
                    game.changePlayer()
            # Get score for the current game and append it to a score list.
            scorelist.append(score)
        # Here, we compute the average number of games won by player X. 
        # If the 'score' value returned is 1, then X has won the game. Otherwise, it's either a draw or a loss, in which
        # case, 0 or -1 is returned respectively.
        # We then divide the number of times X has won (number of occurances of 1 in scorelist) with the total 
        # number of games played which is n.  
        average = (scorelist.count(1)/n) * 100 # We multiply by 100 to express the fraction as a percentage.
        # Print the results.
        print('In', n, ' games, player X has won', average, "% of the times against player Y." )
        print()


Let's play 50 Tic-Tac-Toe games against this opponent, with negamaxIDSab. 

In [41]:
winRate(opponent3, 10, [negamaxIDS, negamaxIDSab], 50)


Now we are trying out function:  negamaxIDS
In 50  games, player X has won 98.0 % of the times against player Y.


Now we are trying out function:  negamaxIDSab
In 50  games, player X has won 96.0 % of the times against player Y.

